In [1]:
!pip install -q transformers[torch] datasets
!pip install -q bitsandbytes trl peft
!pip install flash-attn --no-build-isolation
!pip install -U transformers

In [2]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset as Ds
from datasets import *

raw_datasets = load_dataset("parquet",data_files={'train': '/kaggle/input/final-data/train.parquet', 'test': '/kaggle/input/final-data/test.parquet'})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'messages'],
        num_rows: 1603
    })
    test: Dataset({
        features: ['prompt', 'messages'],
        num_rows: 100
    })
})

In [3]:
example = raw_datasets["train"][0]
messages = example["messages"]

In [4]:
raw_datasets["train"][:2]['messages']

[[{'content': '"DEDAŞ B- 2020-7241 1814482555.pdf\n yapı ve kredi bankası a.ş.www.yapikredi.com.trticaret sicil numarası: 32736mersis no: 0937002089200741işletmenin merkezi: yapıkredi plazad blok 34330 levent - istanbultel: (0212) 339 70 00faks: (0212) 339 60 001/1 istanbul, 16/10/2023yazı no: 23 /opy-mby/1617606556yapı ve kredi bankası a.şbankacılık üssüçayırova 41420 kocaeli - türkiyetel: (0262) 647 10 00faks: (0262) 647 15 47www.yapikredi.com.tr t.c.batmanicra dairesiilgi : 16/10/2023 tarih ve 2020/7241 nolu yazınız,ilgide kayıtlı yazınızda adı geçen özal dürüst adına manisa sanayi sitesi şubesi nezdinde bulunan 97787516 tl numaralı hesabın 2679.18 tl bakiyesine (dosya borcu kadar) kurumunuz adına haciz tatbik edilmiştir.cevaben ve itirazen bildiririz,bilgilerinize arz ederiz.saygılarımızla,yapı ve kredi bankası a.ş.genel müdürlüğü1617606556 - u0t15644"\n verilen metin içerisinden dosya adı,cevap tarihi,icra dairesi,banka adı ve bakiye listesi bilgilerini çıkart.',
   'role': 'user'

In [5]:
example = raw_datasets["train"][0]
messages = example["messages"]
for message in messages:
  role = message["role"]
  content = message["content"]
  print('{0:20}:  {1}'.format(role, content))

user                :  "DEDAŞ B- 2020-7241 1814482555.pdf
 yapı ve kredi bankası a.ş.www.yapikredi.com.trticaret sicil numarası: 32736mersis no: 0937002089200741işletmenin merkezi: yapıkredi plazad blok 34330 levent - istanbultel: (0212) 339 70 00faks: (0212) 339 60 001/1 istanbul, 16/10/2023yazı no: 23 /opy-mby/1617606556yapı ve kredi bankası a.şbankacılık üssüçayırova 41420 kocaeli - türkiyetel: (0262) 647 10 00faks: (0262) 647 15 47www.yapikredi.com.tr t.c.batmanicra dairesiilgi : 16/10/2023 tarih ve 2020/7241 nolu yazınız,ilgide kayıtlı yazınızda adı geçen özal dürüst adına manisa sanayi sitesi şubesi nezdinde bulunan 97787516 tl numaralı hesabın 2679.18 tl bakiyesine (dosya borcu kadar) kurumunuz adına haciz tatbik edilmiştir.cevaben ve itirazen bildiririz,bilgilerinize arz ederiz.saygılarımızla,yapı ve kredi bankası a.ş.genel müdürlüğü1617606556 - u0t15644"
 verilen metin içerisinden dosya adı,cevap tarihi,icra dairesi,banka adı ve bakiye listesi bilgilerini çıkart.
assistant    

In [6]:

from transformers import AutoTokenizer

model_id = "ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [7]:

import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

column_names = list(raw_datasets["train"].features)
raw_datasets = raw_datasets.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

# create the splits
train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["test"]

for index in random.sample(range(len(raw_datasets["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_datasets['train'][index]['text']}")

Sample 317 of the processed training set:

<|system|>
<|eot_id|>
<|user|>
"DEDAŞ M- Kızıltepe İcra 2018-4431 638285632175988165.pdf
 yapı ve kredi bankası a.ş.www.yapikredi.com.trticaret sicil numarası: 32736mersis no: 0937002089200741işletmenin merkezi: yapıkredi plazad blok 34330 levent - istanbultel: (0212) 339 70 00faks: (0212) 339 60 001/1 istanbul, 06/02/2022yazı no: 22 /opy-mby/1371765136yapı ve kredi bankası a.şbankacılık üssüçayırova 41420 kocaeli - türkiyetel: (0262) 647 10 00faks: (0262) 647 15 47www.yapikredi.com.tr t.c.mardinkızıltepe icra dairesiilgi : 04/02/2022 tarih ve 2018/4431 nolu yazınız,ilgide kayıtlı yazınızda adı geçen fevzi işikhan adına ofis şubesi nezdinde 71521704 tl numaralı 0 tl bakiyeli hesabı tespit edilmiştir. bu hesap üzerinde yasal takyidat kayıtları mevcuttur. hacziniz iş bu yasal takyidatlardan sonra gelmek üzere ilgili hesap üzerine kayıtlarımıza işlenmiştir.cevaben ve itirazen bildiririz,saygılarımızla,yapı ve kredi bankası a.ş.genel müdürlüğü1371

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1603
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100
    })
})

In [9]:
from transformers import BitsAndBytesConfig
import torch

# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)
device_map = "auto"#{"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
#     attn_implementation=False,#"flash_attention_2", # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False, # set to False as we're going to use gradient checkpointing
    device_map=device_map,
    quantization_config=quantization_config,
)

In [10]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments



# path where the Trainer will save its checkpoints and logs
trained_model_id = "TUR4N"
output_dir = 'kaggle/working/' + trained_model_id

# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16 else fp16
    bf16=False,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=2,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=8, # originally set to 8
    per_device_train_batch_size=8, # originally set to 8
    #push_to_hub=True,
    #hub_model_id=trained_model_id,
    # hub_strategy="every_save",
    # report_to="tensorboard",
    report_to="none",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)
# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

torch.cuda.empty_cache()

trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:158: UserWarning: You passed `model_init_kwargs` to the SFTTrainer, the value 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradS

In [11]:
train_result = trainer.train()

***** Running training *****
  Num examples = 437
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 110
  Number of trainable parameters = 54,525,952


OutOfMemoryError: CUDA out of memory. Tried to allocate 896.00 MiB. GPU 1 has a total capacity of 14.74 GiB of which 266.12 MiB is free. Process 30050 has 14.48 GiB memory in use. Of the allocated memory 12.71 GiB is allocated by PyTorch, and 1.64 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.save_model(output_dir)

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

In [ ]:
import torch
import transformers

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "Sen bir yapay zeka asistanısın. Kullanıcı sana bir görev verecek. Amacın görevi olabildiğince sadık bir şekilde tamamlamak. Görevi yerine getirirken adım adım düşün ve adımlarını gerekçelendir.",
    },
    {"role": "user", "content": "'DEDAŞ U- 2021-23069 638285075731527975.pdf\
 yapı ve kredi bankası a.ş.www.yapikredi.com.trticaret sicil numarası: 32736mersis no: 0937002089200741işletmenin merkezi: yapıkredi plazad blok 34330 levent - istanbultel: (0212) 339 70 00faks: (0212) 339 60 001/1 istanbul,\
     12/04/2022yazı no: 22 /opy-mby/1410474250yapı ve kredi bankası a.şbankacılık üssüçayırova 41420 kocaeli - türkiyetel: (0262) 647 10 00faks: (0262) 647 15 47www.yapikredi.com.tr t.c.şanliurfa2. icra dairesiilgi : \
     11/04/2022 tarih ve 2021/23069 nolu yazınız,ilgide kayıtlı yazınızda adı geçen mahmut ketboğa adına tüm şubelerimiz nezdinde 12/04/2022 tarih, 17:07 saat itibariyle herhangi bir hak ve alacağı bulunmadığından hacziniz tatbik edilememiştir.\
     diğer yandan bilindiği üzere haciz kararları çerçevesinde 3. kişinin sorumluluğu haciz bildiriminin kendisine tebliğ edildiği tarihteki mevcut durumu ile sınırlı olup, ileride doğacak veya doğması muhtemel bir hakkın 3. kişi tarafından bilinmesi mümkün\
     olmadığından ; ayrıca haciz kararları borçlunun 3. şahıs nezdindeki mevcut ve bilinen alacakları üzerine uygulanabileceğinden gelecekte doğup doğmayacağı bilinmeyen, ümit ve ihtimale dayalı hak ve alacak üzerinde haciz tatbiki mümkün bulunmamaktadır.\
     açıklanan nedenler ile ilgide kayıtlı haciz kararınıza itiraz ediyoruz. ayrıca, ilgide kayıtlı yazınıza cevap verildiği andan itibaren ileriye yönelik herhangi bir takip yükümlülüğümüz olmadığı hususunu, bilgilerinize arz ederiz.cevaben ve itirazen\
     bildiririz,saygılarımızla,yapı ve kredi bankası a.ş.genel müdürlüğü1410474250 - u005908a'\
     verilen metin içerisinden dosya adı,cevap tarihi,icra dairesi,banka adı ve bakiye listesi bilgilerini çıkart."},
]


# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True,return_tensors="pt").to(model.device)

# inference
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])